# Insurance Structures and Visualization

## Overview
- **What this notebook does:** Demonstrates multi-layer insurance program design, layer optimization, premium sensitivity analysis, and technical visualizations of insurance structures (correlation, premium decomposition, capital efficiency).
- **Prerequisites:** [core/01_loss_distributions.ipynb](01_loss_distributions.ipynb)
- **Estimated runtime:** 1--3 minutes
- **Audience:** [Practitioner]

## Why Multi-Layer Structures?
Real-world insurance programs are built from multiple layers, each with its own attachment point, limit, and premium rate. This notebook shows how to design, analyze, and optimize these structures, and includes technical visualizations of correlation structure, premium loading decomposition, and capital efficiency frontiers.

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import warnings
warnings.filterwarnings('ignore')

from ergodic_insurance import InsuranceProgram, EnhancedInsuranceLayer
from ergodic_insurance.loss_distributions import ManufacturingLossGenerator
from ergodic_insurance.visualization import WSJ_COLORS, format_currency
from ergodic_insurance.visualization.technical_plots import (
    plot_correlation_structure,
    plot_premium_decomposition,
    plot_capital_efficiency_frontier_3d,
)

pio.templates.default = "plotly_white"

# Reproducibility
np.random.seed(42)

## Configuration

In [ ]:
REVENUE = 10_000_000
N_SIMULATIONS = 1_000

LOSS_GENERATOR = ManufacturingLossGenerator(
    attritional_params={'base_frequency': 5.0, 'severity_mean': 50_000, 'severity_cv': 0.8},
    large_params={'base_frequency': 0.5, 'severity_mean': 2_000_000, 'severity_cv': 1.2},
    catastrophic_params={'base_frequency': 0.02, 'severity_xm': 10_000_000, 'severity_alpha': 2.5},
    seed=42,
)

# Default three-layer insurance program
DEFAULT_LAYERS = [
    EnhancedInsuranceLayer(0, 5_000_000, 0.015),
    EnhancedInsuranceLayer(5_000_000, 20_000_000, 0.008),
    EnhancedInsuranceLayer(25_000_000, 25_000_000, 0.004),
]

print("Insurance structures notebook configured.")

## 1. Layer Structure Analysis

Analyze how a three-layer program distributes recoveries across layers.

In [ ]:
program = InsuranceProgram(DEFAULT_LAYERS)
results = []
layer_utilization = {i: [] for i in range(3)}

for _ in range(N_SIMULATIONS):
    events, _ = LOSS_GENERATOR.generate_losses(duration=1.0, revenue=REVENUE)
    total_loss = sum(e.amount for e in events)
    total_recovery = 0
    layer_recoveries = [0, 0, 0]

    for event in events:
        details = program.process_claim(event.amount)
        total_recovery += details['insurance_recovery']
        for layer_info in details['layers_triggered']:
            layer_recoveries[layer_info['layer_index']] += layer_info['payment']

    results.append({'total_loss': total_loss, 'total_recovery': total_recovery,
                    'retained': total_loss - total_recovery})
    for i in range(3):
        layer_utilization[i].append(layer_recoveries[i])

res_df = pd.DataFrame(results)

total_premium = program.calculate_annual_premium()
print(f"Insurance Program Summary")
print(f"={'=' * 60}")
print(f"Total Annual Premium: ${total_premium:,.0f}")
print(f"Average Annual Recovery: ${res_df['total_recovery'].mean():,.0f}")
print(f"Average Annual Retention: ${res_df['retained'].mean():,.0f}")
print(f"Loss Ratio: {100 * res_df['total_recovery'].mean() / total_premium:.1f}%")

print(f"\nLayer Utilization:")
for i in range(3):
    util = layer_utilization[i]
    print(f"  Layer {i+1}: Avg recovery ${np.mean(util):,.0f}, "
          f"Triggered {100 * (np.array(util) > 0).mean():.1f}% of years")

## 2. Layer Configuration Optimization

Compare conservative, balanced, aggressive, and data-driven configurations.

In [ ]:
# Generate loss scenarios for optimization
annual_losses = []
for _ in range(N_SIMULATIONS):
    events, stats = LOSS_GENERATOR.generate_losses(duration=1.0, revenue=REVENUE)
    annual_losses.append(stats['total_amount'])
annual_losses = np.array(annual_losses)

p90 = np.percentile(annual_losses, 90)
p99 = np.percentile(annual_losses, 99)
p995 = np.percentile(annual_losses, 99.5)

configurations = [
    {'name': 'Conservative', 'attachments': [0, 1_000_000, 5_000_000],
     'limits': [1_000_000, 4_000_000, 20_000_000], 'rates': [0.025, 0.015, 0.008]},
    {'name': 'Balanced', 'attachments': [0, 5_000_000, 25_000_000],
     'limits': [5_000_000, 20_000_000, 25_000_000], 'rates': [0.015, 0.008, 0.004]},
    {'name': 'Aggressive', 'attachments': [1_000_000, 10_000_000, 50_000_000],
     'limits': [9_000_000, 40_000_000, 50_000_000], 'rates': [0.012, 0.006, 0.002]},
    {'name': 'Data-Driven', 'attachments': [0, p90, p99],
     'limits': [p90, p99 - p90, p995 - p99], 'rates': [0.018, 0.009, 0.003]},
]

opt_results = []
for config in configurations:
    layers = [EnhancedInsuranceLayer(config['attachments'][i], config['limits'][i], config['rates'][i])
              for i in range(3)]
    prog = InsuranceProgram(layers)
    premium = prog.calculate_annual_premium()
    recoveries = []
    for loss in annual_losses:
        details = prog.process_claim(loss)
        recoveries.append(details['insurance_recovery'])

    opt_results.append({
        'Configuration': config['name'], 'Premium': premium,
        'Avg Recovery': np.mean(recoveries),
        'Avg Retention': np.mean(annual_losses) - np.mean(recoveries),
        'Loss Ratio': np.mean(recoveries) / premium if premium > 0 else 0,
        'Efficiency': np.mean(recoveries) / np.mean(annual_losses),
    })

opt_df = pd.DataFrame(opt_results)
print("Layer Configuration Comparison:")
print(opt_df.to_string(index=False))
print(f"\nBest efficiency: {opt_df.loc[opt_df['Efficiency'].idxmax(), 'Configuration']}")

## 3. Premium Sensitivity Analysis

How sensitive are net benefits to changes in premium rates?

In [ ]:
base_rates = [0.015, 0.008, 0.004]
base_attachments = [0, 5_000_000, 25_000_000]
base_limits = [5_000_000, 20_000_000, 25_000_000]

multipliers = np.linspace(0.5, 2.0, 16)
sens_results = []

for mult in multipliers:
    adjusted_rates = [r * mult for r in base_rates]
    layers = [EnhancedInsuranceLayer(base_attachments[i], base_limits[i], adjusted_rates[i])
              for i in range(3)]
    prog = InsuranceProgram(layers)
    premium = prog.calculate_annual_premium()
    recoveries = [prog.process_claim(loss)['insurance_recovery'] for loss in annual_losses]
    sens_results.append({
        'multiplier': mult, 'premium': premium,
        'avg_recovery': np.mean(recoveries),
        'loss_ratio': np.mean(recoveries) / premium if premium > 0 else 0,
        'net_benefit': np.mean(recoveries) - premium,
    })

sens_df = pd.DataFrame(sens_results)
optimal_mult = sens_df.loc[sens_df['net_benefit'].idxmax(), 'multiplier']

fig = make_subplots(rows=1, cols=2, subplot_titles=('Premium vs Recovery', 'Net Benefit'))

fig.add_trace(go.Scatter(x=sens_df['multiplier'], y=sens_df['premium'], mode='lines',
                         name='Premium', line=dict(color=WSJ_COLORS['blue'], width=2)), row=1, col=1)
fig.add_trace(go.Scatter(x=sens_df['multiplier'], y=sens_df['avg_recovery'], mode='lines',
                         name='Avg Recovery', line=dict(color=WSJ_COLORS['orange'], width=2, dash='dash')),
              row=1, col=1)

colors = ['green' if x > 0 else 'red' for x in sens_df['net_benefit']]
fig.add_trace(go.Bar(x=sens_df['multiplier'], y=sens_df['net_benefit'],
                     marker_color=colors), row=1, col=2)

fig.update_layout(height=400, title_text='Premium Rate Sensitivity Analysis')
fig.update_xaxes(title_text='Rate Multiplier')
fig.update_yaxes(title_text='Amount', tickformat='$.2s', row=1, col=1)
fig.update_yaxes(title_text='Net Benefit', tickformat='$.2s', row=1, col=2)
fig.show()

print(f"Optimal rate multiplier: {optimal_mult:.2f}x")

## 4. Risk Correlation Structure (Figure B2)

Visualize correlations between operational and financial risks.

In [ ]:
n_samples = 1_000
operational_data = np.exp(np.random.multivariate_normal(
    [0, 0, 0], [[1.0, 0.6, 0.3], [0.6, 1.0, 0.4], [0.3, 0.4, 1.0]], n_samples)) * 10_000
financial_data = np.exp(np.random.multivariate_normal(
    [0, 0, 0], [[1.0, 0.8, 0.5], [0.8, 1.0, 0.7], [0.5, 0.7, 1.0]], n_samples)) * 50_000

risk_data = {"Operational": operational_data, "Financial": financial_data}

fig = plot_correlation_structure(
    risk_data, correlation_type="pearson",
    title="Risk Correlation Structure Analysis", figsize=(16, 10), show_copula=True,
)
plt.show()

## 5. Premium Loading Decomposition (Figure C4)

Break down premiums into expected loss, volatility load, tail load, expenses, and profit margin.

In [ ]:
premium_components = {
    "Small Company": {
        "Primary ($0-5M)": {
            "expected_loss": 250_000, "volatility_load": 75_000, "tail_load": 50_000,
            "expense_load": 87_500, "profit_margin": 37_500,
        },
        "Excess ($5-15M)": {
            "expected_loss": 100_000, "volatility_load": 40_000, "tail_load": 35_000,
            "expense_load": 35_000, "profit_margin": 15_000,
        },
    },
    "Large Company": {
        "Primary ($0-10M)": {
            "expected_loss": 800_000, "volatility_load": 120_000, "tail_load": 100_000,
            "expense_load": 150_000, "profit_margin": 80_000,
        },
        "Excess ($10-25M)": {
            "expected_loss": 300_000, "volatility_load": 75_000, "tail_load": 75_000,
            "expense_load": 60_000, "profit_margin": 40_000,
        },
    },
}

fig = plot_premium_decomposition(
    premium_components,
    title="Premium Loading Decomposition by Company Size and Layer",
    figsize=(14, 8), show_percentages=True,
)
plt.show()

print("Expected loss typically represents 40-60% of premium.")
print("Volatility and tail loads increase for higher layers.")
print("Expense ratios decrease with company size (economies of scale).")

## 6. Capital Efficiency Frontier (Figure C5)

3D visualization showing the trade-off between ROE, ruin probability, and insurance spend.

In [ ]:
def generate_efficiency_surface(n_ruin=25, n_spend=30, base_roe=0.12, spend_max=1.5e6):
    ruin_probs = np.linspace(0.001, 0.15, n_ruin)
    insurance_spends = np.linspace(0, spend_max, n_spend)
    roe_surface = np.zeros((n_ruin, n_spend))
    for i, ruin in enumerate(ruin_probs):
        for j, spend in enumerate(insurance_spends):
            roe = base_roe * (1 - ruin * 2) - (spend / 10e6) * 0.5
            roe += min(spend / spend_max, 1) * 0.08 * (1 - ruin)
            roe += np.random.normal(0, 0.005)
            roe_surface[i, j] = max(roe, 0)
    return {"roe": roe_surface, "ruin_prob": ruin_probs, "insurance_spend": insurance_spends}

efficiency_data = {"Small": generate_efficiency_surface()}

# Optimal path
n_pts = 20
optimal_paths = {"Small": np.column_stack([
    np.linspace(0.10, 0.01, n_pts),
    np.linspace(0.2e6, 1.0e6, n_pts),
    np.linspace(0.08, 0.11, n_pts) + 0.02 * np.sin(np.pi * np.linspace(0, 1, n_pts)),
])}

fig = plot_capital_efficiency_frontier_3d(
    efficiency_data, optimal_paths=optimal_paths,
    title="Capital Efficiency Frontier: ROE vs Ruin vs Insurance Spend",
    figsize=(14, 10), view_angles=(25, 45),
)
plt.show()

print("The optimal path (red line) shows the trade-off between insurance cost and risk reduction.")
print("More insurance can actually improve ROE when risk-adjusted.")

## Key Takeaways

- Multi-layer programs provide more efficient risk transfer than single large layers.
- Data-driven attachment points (aligned with loss distribution percentiles) outperform ad-hoc structures.
- Premium sensitivity analysis reveals diminishing returns beyond certain rate levels.
- Correlation analysis is critical for understanding aggregate risk exposures.
- The capital efficiency frontier shows that insurance can enhance ROE when properly structured.

## Next Steps

- **See the ergodic advantage:** [core/03_ergodic_advantage.ipynb](03_ergodic_advantage.ipynb)
- **Run large-scale Monte Carlo:** [core/04_monte_carlo_simulation.ipynb](04_monte_carlo_simulation.ipynb)
- **Optimize insurance programs:** [optimization/01_retention_optimization.ipynb](../optimization/01_retention_optimization.ipynb)